<a href="https://colab.research.google.com/github/kbdev20/Convolution_Crusaders/blob/main/Convolution_Crusaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import zipfile
import os


# Step 2: Specify the zip file name and extraction directory
zip_file_path = '/content/drive/MyDrive/dataa/airbus-ship-detection.zip'
extraction_directory = '/content/drive/MyDrive/dataa/'

# Ensure the extraction directory exists
if not os.path.exists(extraction_directory):
    os.makedirs(extraction_directory)

# Step 3: Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_directory)

print(f"Files extracted to {extraction_directory}")


In [ ]:
import pandas as pd
import numpy as np
from skimage.measure import label, regionprops

# Load the RLE-encoded CSV
file_path = '/content/drive/MyDrive/dataa/train_ship_segmentations_v2.csv'
df = pd.read_csv(file_path).dropna()  # dropna() to remove rows with NaN values

# Function to decode RLE
def rle_decode(mask_rle, shape=(768, 768)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

# Function to convert binary mask to YOLO format bounding box
def mask_to_yolo_bbox(mask):
    labeled = label(mask)
    props = regionprops(labeled)
    if not props:
        return None
    y1, x1, y2, x2 = props[0].bbox
    height, width = mask.shape
    x_center = (x1 + x2) / 2 / width
    y_center = (y1 + y2) / 2 / height
    bbox_width = (x2 - x1) / width
    bbox_height = (y2 - y1) / height
    return [x_center, y_center, bbox_width, bbox_height]

# Apply the conversions
df['bbox'] = df['EncodedPixels'].apply(lambda x: mask_to_yolo_bbox(rle_decode(x)))

# Remove rows with None values (if any)
df = df.dropna()

# Create a new DataFrame with YOLO format annotations
yolo_df = pd.DataFrame()
yolo_df['ImageId'] = df['ImageId']
yolo_df[['x_center', 'y_center', 'width', 'height']] = pd.DataFrame(df['bbox'].tolist(), index=df.index)

# Print the new DataFrame
print(yolo_df.head())


In [ ]:
!pip install scikit-learn


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming yolo_df is your DataFrame and it's already defined and filled with your data
# We'll split it into training, validation, and test sets

# First, let's split the data into training (70%) and temp (30%)
train_df, temp_df = train_test_split(yolo_df, test_size=0.3, random_state=42)

# Then, let's split the temp data into validation (50%) and test (50%)
# So, validation and test sets will each be 15% of the original data
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Print the shapes of the datasets
print(f"Train: {train_df.shape}, Validation: {val_df.shape}, Test: {test_df.shape}")


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

# Assuming yolo_df is your DataFrame with ['ImageId', 'x_center', 'y_center', 'width', 'height'] columns
# and it's already defined and filled with your data

# Directory containing the images
image_dir = '/content/drive/MyDrive/dataa/train_v2/'

# Function to convert YOLO format bounding box to matplotlib format [x, y, width, height]
def yolo_to_matplotlib_bbox(yolo_bbox, image_shape):
    x_center, y_center, width, height = yolo_bbox
    x_center *= image_shape[1]
    y_center *= image_shape[0]
    width *= image_shape[1]
    height *= image_shape[0]
    x = x_center - width / 2
    y = y_center - height / 2
    return [x, y, width, height]

# Plot the first few images with bounding boxes
num_images_to_plot = 5
for i, row in yolo_df.head(num_images_to_plot).iterrows():
    image_path = image_dir + row['ImageId']
    image = Image.open(image_path)
    image_shape = image.size

    # Create a matplotlib figure and axes
    fig, ax = plt.subplots(1)

    # Display the image
    ax.imshow(image)

    # Convert YOLO bounding box to matplotlib format and draw it
    yolo_bbox = [row['x_center'], row['y_center'], row['width'], row['height']]
    x, y, width, height = yolo_to_matplotlib_bbox(yolo_bbox, image_shape)
    rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

    # Show the plot
    plt.show()
